# 1. Fine-tuning a model on the YNAT Task

- [Text Classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=71pt6N0eIrJo)을 KLUE의 주제 분류 데이터셋 YNAT에 맞게 수정 및 번역한 자료입니다.
- 다루는 내용은 다음과 같습니다.
  - HuggingFace Datasets을 활용하여 KLUE 데이터셋 쉽게 전처리하기
  - HuggingFace Hub에서 사전학습된 언어 모델을 다운로드 받아 사용하고, 학습한 모델을 업로드하여 공유하기
  - `Trainer` 객체를 사용하여 모델 학습 및 평가 & hyperparameter search 하기
  - Weights & Biases 를 활용하여 실험 관리하기

<br>

## Install packages

In [1]:
!pip install transformers datasets wandb

     |████████████████████████████████| 2.8 MB 9.9 MB/s 
     |████████████████████████████████| 270 kB 73.5 MB/s 
     |████████████████████████████████| 1.7 MB 57.1 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
     |████████████████████████████████| 636 kB 58.7 MB/s 
     |████████████████████████████████| 243 kB 64.5 MB/s 
     |████████████████████████████████| 123 kB 72.4 MB/s 
     |████████████████████████████████| 1.3 MB 54.8 MB/s 
     |████████████████████████████████| 97 kB 7.3 MB/s 
     |████████████████████████████████| 180 kB 64.2 MB/s 
     |████████████████████████████████| 139 kB 72.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 294 kB 70.7 MB/s 
     |████████████████████████████████| 142 kB 72.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl 

In [2]:
import transformers

print(transformers.__version__)

4.10.3


<br>

## Hugging Face Settings

- 학습한 모델을 커뮤니티에서 공유하거나, 다른 pretrained model처럼 쉽게 불러와 사용하고 싶다면 다음 과정을 수행해야 한다.
  1. Hugging Face 웹사이트로부터 얻은 자신의 authentication token을 저장
  2. 아래 셀들을 실행하여 아이디와 비밀번호 입력

In [3]:
!git config --global credential.helper store

In [4]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: zgotter
Password: 
Login successful
Your token has been saved to /root/.huggingface/token


<br>

- 그 다음 Git-LFS를 설치하고 Git을 설정해야 한다.
- 다음 셀을 실행하여 e-mail과 이름을 설정한다.

In [5]:
!pip install hf-lfs
!git config --global user.email "shkim4738@gmail.com"
!git config --global user.name "zgotter"

     |████████████████████████████████| 4.0 MB 9.2 MB/s 


<br>

## Argument Setting

In [6]:
task = 'ynat'
model_checkpoint = 'klue/bert-base'
batch_size = 256

<br>

## 1.1 Loading the Dataset

- [🤗 Datasets](https://github.com/huggingface/datasets) 라이브러리를 사용해 데이터셋을 다운로드 합니다.
- `load_dataset`을 이용하면 쉽게 다운로드 할 수 있습니다.

In [7]:
from datasets import load_dataset

dataset = load_dataset('klue', task)

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.93M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/ynat/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# 샘플 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [9]:
dataset['train'][5]

{'date': '2016.07.18. 오전 9:46',
 'guid': 'ynat-v1_train_00005',
 'label': 0,
 'title': '야외서 생방송 하세요…액션캠 전용 요금제 잇따라',
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=226&oid=001&aid=0008547867'}

<br>

- 데이터셋이 어떻게 생겼는 지 한 눈에 확인해보자.
- 아래 함수는 데이터셋에서 임의의 샘플을 추출해 확인한다.

In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(dataset['train'])

,guid,title,label,url,date
0,ynat-v1_train_17129,경자년 맞아 경자님께 선물 드려요…KT 해외여행 로밍 이벤트,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=731&oid=001&aid=0011306768,2019.12.31. 오전 9:50
1,ynat-v1_train_33818,북한 한일 위안부 합의 미국이 막후조종 주장,정치,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=268&oid=001&aid=0008157609,2016.02.01. 오전 8:26
2,ynat-v1_train_19137,민속예술 발굴과 재현의 장…김해서 한국민속예술축제,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0009558694,2017.09.20. 오전 11:09
3,ynat-v1_train_42593,첼리스트 양성원 바흐 연주할 땐 발가벗은 느낌,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0009507188,2017.08.29. 오후 5:18
4,ynat-v1_train_44138,신간 집 짓는 사람·나와 마주하는 시간,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0010751623,2019.04.09. 오후 4:24
5,ynat-v1_train_43716,NYT 힐러리 대통령 가능성 83%…공화당 텃밭서도 접전,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=232&oid=001&aid=0008595184,2016.08.08. 오전 11:08
6,ynat-v1_train_02454,당정청 오늘 국가교육위원회 설치 방안 협의,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010687784,2019.03.12. 오전 5:15
7,ynat-v1_train_28269,SC제일·KDB생명·롯데손보 소비자보호 낙제점,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258&oid=001&aid=0010316465,2018.09.02. 오후 12:00
8,ynat-v1_train_10297,롯데 손아섭 야구부 후배들에게 2억원 상당 용품 지원,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0009964866,2018.03.18 14:15
9,ynat-v1_train_20750,나경원 경제청문회 먼저하고 추경심사 돌입해야…與에 통첩종합,정치,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010894075,2019.06.16. 오후 12:33


<br>

## 1.2 Processing the data

- Transformers `Tokenizer`는 모델의 입력 텍스트를 토크나이징(and 각 토큰들에 할당된 IDs로 변환하는 작업도 함께 수행)하고, 모델이 입력받는 포맷으로 만들어준다.
  - 각 모델이 입력받아야 할 입력 포맷
  - BERT Pre-training의 경우 attention mask, token type ids 까지 함께 처리

<br>

### 1.2.1 `AutoTokenizer`

- 이 모든 작업을 하기 위해서 먼저 `AutoTokenizer.from_pretrained()`를 사용해 토크나이저를 준비해야 한다.
- 이걸 사용하면 다음과 같은 것을 할 수 있다.
  - 사용하고자 하는 모델과 관련된 토크나이저를 얻을 수 있음
  - 특정 체크포인트를 pre-training할 때 사용된 vocabulary를 다운로드할 수 있음
    - vocabulary는 캐시되므로, 다음번에 셀을 실행할 때 다시 다운로드 되지 않는다.

In [12]:
model_checkpoint

'klue/bert-base'

In [13]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/428 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

<br>

### 1.2.2 Examples

- 토크나이저를 이용해 문장을 인코딩할 수 있다.
- 하나의 문장을 입력하거나 문장 쌍을 입력할 수 있다.

In [14]:
sent1 = "가짜연구소 3기 KLUE 벤치마크 팀 화이팅!!!"
sent2 = "잘 부탁 드려요"

In [15]:
tokenizer(sent1, sent2)

{'input_ids': [2, 8711, 19648, 23, 2015, 47, 2237, 2309, 2105, 9262, 9112, 1823, 21121, 5, 5, 5, 3, 1521, 5527, 20469, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
print(tokenizer(sent1).tokens())

['[CLS]', '가짜', '##연구소', '3', '##기', 'K', '##L', '##U', '##E', '벤치', '##마크', '팀', '화이팅', '!', '!', '!', '[SEP]']


In [17]:
print(tokenizer(sent1, sent2).tokens())

['[CLS]', '가짜', '##연구소', '3', '##기', 'K', '##L', '##U', '##E', '벤치', '##마크', '팀', '화이팅', '!', '!', '!', '[SEP]', '잘', '부탁', '드려요', '[SEP]']


<br>

- YNAT task에서는 분류 대상 문장이 포함된 열인 `title`을 모델 입력으로 사용한다.

In [18]:
print(f"Sentence: {dataset['train'][0]['title']}")

Sentence: 유튜브 내달 2일까지 크리에이터 지원 공간 운영


<br>

### 1.2.3 Define preprocessing function

- 전처리를 위해 사용할 함수를 작성해보자.
- `tokenizer`의 입력으로 `dataset`의 `text`열을 입력하면서 `truncation=True`를 설정하면 된다.
  - `truncation=True`: 모델이 입력 받을 수 있는 토큰 최대 길이를 벗어난 토큰들을 잘라주는 옵션

In [19]:
def preprocess_function(examples):
    return tokenizer(examples['title'], truncation=True)

<br>

- 이 함수는 입력 데이터를 하나 이상 받을 수 있어 배치로 처리 가능하다.
- 이 경우 각각의 key의 value는 리스트 형태로 반환된다.

In [20]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[2, 10637, 8474, 22, 2210, 2299, 2118, 28940, 3691, 4101, 3792, 3], [2, 24905, 1042, 4795, 19982, 2129, 121, 6904, 16311, 1, 14392, 3], [2, 4172, 3797, 3728, 2107, 2134, 3777, 904, 6022, 2332, 2113, 2259, 4523, 1380, 2259, 2062, 3], [2, 12417, 2155, 7840, 604, 2859, 3873, 11554, 2522, 1539, 2073, 8446, 6626, 18818, 575, 3], [2, 13203, 2179, 2366, 4197, 7551, 2096, 8542, 2088, 2353, 886, 1244, 4393, 2027, 22, 2207, 8189, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

<br>

### 1.2.4 Apply preprocessing function

- 이 함수를 전체 데이터셋의 문장에 적용시키기 위해서, 앞서 생성한 `dataset` 객체의 `map` 메서드를 사용한다.
- 이 떄 `dataset`의 모든 splits에 적용되어 한 번의 실행 만으로 "train", "validation" 셋이 모두 모델의 입력으로 변환된다.

In [21]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [22]:
encoded_dataset['train'][5]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'date': '2016.07.18. 오전 9:46',
 'guid': 'ynat-v1_train_00005',
 'input_ids': [2,
  8296,
  2112,
  28814,
  1889,
  5971,
  121,
  8765,
  2822,
  5119,
  10841,
  7912,
  3],
 'label': 0,
 'title': '야외서 생방송 하세요…액션캠 전용 요금제 잇따라',
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=226&oid=001&aid=0008547867'}

In [23]:
encoded_dataset['validation'][5]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'date': '2020.02.24. 오후 3:40',
 'guid': 'ynat-v1_dev_00005',
 'input_ids': [2, 9731, 17759, 3664, 14943, 11456, 100, 7167, 16391, 16599, 3],
 'label': 2,
 'title': '모의선거 교육 불허 선관위·교육부 각성하라',
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=250&oid=001&aid=0011423921'}

<br>

- 변환 결과는 `Datasets` 라이브러리에 의해 자동으로 캐시되어 다음 번에 노트북을 실행할 때는 캐시된 데이터를 불러와 시간을 절약한다.
- `Datasets` 라이브러리는 `map` 메서드에 입력되는 함수의 수정 여부를 체크하여 캐시된 데이터의 사용여부를 결정한다.
- 이때 `Datasets`에 캐시된 파일의 사용여부를 알려주며, `map` 메서드 호출 시 `load_from_cache_file=False`으로 변환을 강제할 수 있다.

- `batched=True`를 사용해서 배치로 묶어서 텍스트를 인코딩했다.
- 이는 앞서 `AutoTokenizer`에서 `use_fast=True`를 지정하여 로드한 fast tokenizer의 이점을 활용하기 위함이다.
  - fast tokenizer는 멀티 쓰레딩을 사용하여 텍스트를 배치 형태로 거의 동시에 처리한다.

<br>

## 1.3 Fine-tuning the model

- 데이터가 준비되었으니, pre-trained model을 다운 받아 fine-tuning 해보자.

<br>

### 1.3.1 모델 정의

- YNAT은 문장 분류와 관련된 task이다.
- 그러므로 `AutoModelForSequenceClassification` 클래스를 사용한다.
- tokenizer 때처럼, `from_pretrained()` 메서드로 모델을 다운로드 받고 캐시한다.
- 한 가지 설정해줘야 할 것은 label의 개수이다.
  - YNAT은 총 7개의 label로 구성됨

In [24]:
dataset['train'].features['label']

ClassLabel(num_classes=7, names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], names_file=None, id=None)

<br>

- 여기서는 [Model Hub](https://huggingface.co/models)에서 pre-trained된 모델이 존재할 경우 `[모델명]ForPreTraining` 이라는 모델 체크포인트를 다운로드하여 YNAT task를 학습한다.

In [25]:
model_checkpoint

'klue/bert-base'

In [26]:
from transformers import AutoModelForSequenceClassification

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

- 위의 경고 메세지는 다음을 의미한다.
  - `BertForPreTraining`엔 있지만 `BertForSequenceClassification`엔 없는 레이어는 버림
  - `BertForSequenceClassification`엔 있지만 `BertForPreTraining`엔 없는 레이어는 랜덤 초기화
- 즉, `BertForSequenceClassification` 모델을 fine-tuninig 하지 않으면 좋은 성능을 얻지 못할 것이라는 것을 의미한다.
  - reference: [https://github.com/huggingface/transformers/issues/5421](https://github.com/huggingface/transformers/issues/5421)

<br>

### 1.3.2 `Trainer` 객체 활용

- HuggingFace 라이브러리에서는 `Trainer` 객체를 사용하여 복잡한 학습 코드를 짤 필요 없이 편리하게 학습할 수 있다.

<br>

#### 1.3.2.1 `TrainingArgument`

- `Trainer` 객체는 `TrainingArgument`를 입력으로 받는다.
  - `TrainingArgument`: 모델 학습을 위해 설정하는 값들이 attribute로 들어 있는 클래스
- 입력 받는 인자 중 필수 인자로 `output_dir`이 존재한다.
  - `output_dir`: 모델의 checkpoint를 저장하기 위한 경로를 의미

In [27]:
from transformers import TrainingArguments
import os

model_name = model_checkpoint.split('/')[-1]
output_dir = os.path.join('test-klue', 'ynat')
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy='epoch', # or 'steps'
    load_best_model_at_end=True,
    # save_steps=500,

    # Dataset
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5, # or 5e-5
    weight_decay=0.01, # or 0
    # warmup_steps=200,

    # Regularization
    # max_grad_norm=1.0,
    # label_smoothing_factor=0.1,

    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy='epoch',

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}", # bert-base-finetuned-ynat

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42
)

<br>

- `load_best_model_at_end=True`
  - 학습을 끝까지 수행한 모델이 최고 성능을 내는 모델은 아니다.
  - 따라서 학습이 종료되었을 때 `Trainer` 객체가 `metric_for_best_model`에서 설정한 `metric`에 따라 가장 좋은 성능을 보인 모델을 불러오도록 `load_best_model_at_end` 파라미터를 설정한다.

<br>

- `push_to_hub`, `push_to_hub_model_id`
  - 두 파라미터는 학습이 끝날 때 모델을 [Hub](https://huggingface.co/models)에 push 할 떄 설정한다.

<br>

#### 1.3.2.2 metric 계산 방법 정의

- 마지막으로 `Trainer` 세팅을 위해 metric을 계산하는 방법을 정의한다.
  - 먼저 `datasets` 라이브러리에서 F1-score metric을 로드하여 `metric_marcof1` 변수에 할당한다.
  - 그다음 모델의 출력값을 입력으로 받는 `compute_metrics` 함수를 정의하여 `metric_marcof1`을 계산한 결과를 출력한다.
- 입력에 적절한 모델의 출력값을 전처리해주는 과정이 포함된다.

In [28]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트 확인
from datasets import list_metrics

metrics_list = list_metrics()

print(len(metrics_list))
print(', '.join(metric for metric in metrics_list))

28
accuracy, bertscore, bleu, bleurt, cer, comet, coval, cuad, f1, gleu, glue, indic_glue, matthews_correlation, meteor, pearsonr, precision, recall, rouge, sacrebleu, sari, seqeval, spearmanr, squad, squad_v2, super_glue, wer, wiki_split, xnli


<br>

- YNAT의 metric은 F1 score를 사용한다.

In [29]:
from datasets import load_metric

metric_marcof1 = load_metric('f1')

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

<br>

- metric을 계산하는 함수를 정의한다.

In [36]:
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids # ground truth

    return metric_marcof1.compute(
        predictions=predictions,
        references=labels,
        average='macro'
    )

<br>

#### 1.3.2.3 `Trainer` 객체 생성

In [37]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/test-klue/ynat is already a clone of https://huggingface.co/zgotter/bert-base-finetuned-ynat. Make sure you pull the latest changes with `repo.git_pull()`.


- `Trainer`의 입력으로 `tokenizer`를 넣어주는 이유
  - 앞에서 `tokenizer`를 사용해 전처리를 수행했다.
  - 그럼에도 `Trainer`의 입력으로 다시 넣는 이유는 **패딩을 적용해서 입력 샘플들을 동일한 길이로 만들기 위해 사용하기 때문**이다.
    - 데이터 로더의 마지막 과정에서 패딩이 적용된다.
  - 모델에 따라 패딩에 대한 기본 설정이 다르다.
    - 왼쪽 패딩, 오른쪽 패딩, 또는 패딩 인덱스 번호 설정 등
  - 그렇기 때문에 `Trainer`는 이와 관련된 작업을 수행할 수 있는 `tokenizer` 객체를 사용한다.
- `tokenizer` 대신에 `data_collator`를 커스텀하여 `tokenizer`를 적용하였을 때ㅐ와 동일한 형태의 입력을 만들어줄 수도 있다.

<br>

### 1.3.3 wandb 사용 설정

- huggingface는 모델 학습 로그를 기록할 때 Tensorboard 또는 wandb를 사용할 수 있다.

In [38]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

<br>

- 실험 관리를 위해 id값을 생성한다.

In [39]:
id = wandb.util.generate_id()

print(id)

i40gw0vn


<br>

- `wandb.init()` arguments
  - `project`
    - 실험 기록을 관리할 프로젝트 이름
    - 프로젝트가 존재하지 않을 경우 입력받은 이름으로 생성
    - 여기서는 예시로 `klue`로 설정
  - `entity`
    - weights & biases 사용자명 또는 팀 이름
  - `id`
    - 실험에 부여된 고유 아이디
  - `name`
    - 실험에 부여한 이름
  - `resume`
    - 실험을 재개할 때, 실험에 부여한 고유 아이디 입력 여부

In [40]:
wandb.init(
    project='klue-test',
    entity='zgotter',
    id=id,
    name='ynat',
    resume=True
)

wandb: WARNING Tried to auto resume run with id 1xfclmz6 but id i40gw0vn is set.


<br>

### 1.3.4 fine-tuning 실시

In [41]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running training *****
  Num examples = 45678
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 895
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,F1
1,No log,0.422254,0.854944
2,No log,0.370978,0.866912
3,0.257600,0.389137,0.863052
4,0.257600,0.396837,0.861167
5,0.257600,0.404358,0.861729


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat/checkpoint-179
Configuration saved in test-klue/ynat/checkpoint-179/config.json
Model weights saved in test-klue/ynat/checkpoint-179/pytorch_model.bin
tokenizer config file saved in test-klue/ynat/checkpoint-179/tokenizer_config.json
Special tokens file saved in test-klue/ynat/checkpoint-179/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat/checkpoint-358
Configuration saved in test-klue/ynat/checkpoint-358/config.json
Model weights saved in test-klue/

TrainOutput(global_step=895, training_loss=0.22699027461046614, metrics={'train_runtime': 1176.7584, 'train_samples_per_second': 194.084, 'train_steps_per_second': 0.761, 'total_flos': 2846762274066300.0, 'train_loss': 0.22699027461046614, 'epoch': 5.0})

<br>

### 1.3.5 모델 성능 확인

- `evaluate()` 메서드를 사용하여 `Trainer`가 best 모델로 불러온 모델의 성능을 확인해볼 수 있다.

In [43]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256


{'epoch': 5.0,
 'eval_f1': 0.8669116640755216,
 'eval_loss': 0.3709777593612671,
 'eval_runtime': 15.1537,
 'eval_samples_per_second': 600.974,
 'eval_steps_per_second': 2.376}

<br>

### 1.3.6 wandb 종료

In [44]:
wandb.finish()

eval/f1,0.86691
eval/loss,0.37098
eval/runtime,15.1537
eval/samples_per_second,600.974
eval/steps_per_second,2.376
train/epoch,5.0
train/global_step,895
train/learning_rate,1e-05
train/loss,0.2576
train/total_flos,2846762274066300.0
train/train_loss,0.22699


eval/f1,▁█▆▅▅█
eval/loss,█▁▃▅▆▁
eval/runtime,█▃▂▂▃▁
eval/samples_per_second,▁▆▇▇▆█
eval/steps_per_second,▁▆▇▇▆█
train/epoch,▁▃▄▅▆███
train/global_step,▁▃▄▅▆███
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁


<br>

### 1.3.7 Hugging Face Hub에 업로드

- 현재 tokenizer를 비롯한 모델을 Hub에 업로드하려면 `push_to_hub()` 메서드를 사용한다.

In [45]:
trainer.push_to_hub()

Saving model checkpoint to test-klue/ynat
Configuration saved in test-klue/ynat/config.json
Model weights saved in test-klue/ynat/pytorch_model.bin
tokenizer config file saved in test-klue/ynat/tokenizer_config.json
Special tokens file saved in test-klue/ynat/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/422M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.55k/2.55k [00:00<?, ?B/s]

To https://huggingface.co/zgotter/bert-base-finetuned-ynat
   7e2e3e6..ef7436c  main -> main



'https://huggingface.co/zgotter/bert-base-finetuned-ynat/commit/ef7436caa4323e8382054d9c046da05597bc6782'

<br>

- 업로드한 모델을 `Hub-user-name/사용자가 지정한 이름`으로 바로 다운로드하여 사용할 수 있다.

In [46]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('zgotter/bert-base-finetuned-ynat')

https://huggingface.co/zgotter/bert-base-finetuned-ynat/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpsl4btndl


Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

storing https://huggingface.co/zgotter/bert-base-finetuned-ynat/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/e621a652c31ceb34b6ee817a628e6ee86da436cbaa79800a668a93e4b554a641.fe893e5b967b6067a3dead10e587be1007b7a381cde93482a93ec97122a1191d
creating metadata file for /root/.cache/huggingface/transformers/e621a652c31ceb34b6ee817a628e6ee86da436cbaa79800a668a93e4b554a641.fe893e5b967b6067a3dead10e587be1007b7a381cde93482a93ec97122a1191d
loading configuration file https://huggingface.co/zgotter/bert-base-finetuned-ynat/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e621a652c31ceb34b6ee817a628e6ee86da436cbaa79800a668a93e4b554a641.fe893e5b967b6067a3dead10e587be1007b7a381cde93482a93ec97122a1191d
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

storing https://huggingface.co/zgotter/bert-base-finetuned-ynat/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/81e0e8a029f3dd9da62fae18c056e5c43c67c1677fa80e808c99960fbd6bf819.43d706c0296f28b2622cfc3cdf1a711b33e9616751b103f5719826ceae396781
creating metadata file for /root/.cache/huggingface/transformers/81e0e8a029f3dd9da62fae18c056e5c43c67c1677fa80e808c99960fbd6bf819.43d706c0296f28b2622cfc3cdf1a711b33e9616751b103f5719826ceae396781
loading weights file https://huggingface.co/zgotter/bert-base-finetuned-ynat/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/81e0e8a029f3dd9da62fae18c056e5c43c67c1677fa80e808c99960fbd6bf819.43d706c0296f28b2622cfc3cdf1a711b33e9616751b103f5719826ceae396781
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at zgotter/bert-base-finetuned-ynat.
If your task is simila

<br>

## 1.4 Hyperparameter search

- `Trainer`는 [optuna](https://optuna.org/) 또는 [Ray Tune](https://docs.ray.io/en/latest/tune/) 를 이용한 hyperparameter search를 지원한다.

<br>

### 1.4.1 라이브러리 설치

- 이 섹션을 실습하기 위해서는 두 라이브러리 모두 설치되어 있어야 한다.
- 아래 코드를 실행시켜 라이브러리를 설치한 뒤 다음 셀을 실행한다.

In [48]:
!pip install optuna
!pip install ray[tune]

     |████████████████████████████████| 49.6 MB 10 kB/s 
     |████████████████████████████████| 72 kB 705 kB/s 
     |████████████████████████████████| 124 kB 76.1 MB/s 


<br>

### 1.4.2 Argument 세팅

In [49]:
model_checkpoint

'klue/bert-base'

In [50]:
model_name = model_checkpoint.split('/')[-1]
output_dir = os.path.join('test-klue', 'ynat-hps')
logging_dir = os.path.join(output_dir, 'logs')

args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,
    # save_steps = 500,


    # Dataset
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    # warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation 
    metric_for_best_model='eval_f1',
    evaluation_strategy = "epoch",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

PyTorch: setting up devices


<br>

### 1.4.3 모델 정의 함수

- hyperparameter search 동안에  `Trainer`는 학습을 여러 번 실행한다.
- 따라서 모델이 매 학습마다 다시 초기화될 수 있도록 모델이 함수에 의해 정의되도록 한다.

In [51]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

<br>

### 1.4.4 Trainer 객체 생성

- 그 다음 모델을 직접 입력하는 대신 `model_init`으로 초기화하는 함수를 입력한다.
- 나머지는 앞에서 했던 것과 거의 동일하게 `Trainer`를 초기화한다.

<br>

- 전체 데이터셋에 대해서 사용할 때 몇몇 task에서는 시간이 오래 걸릴 수 있다.
- 이럴 땐 아래 예시 코드처럼 `dataset` 객체의 `shard` 메서드를 이용해 데이터셋의 일부(예시에선 1/10)만을 선택하여 hyperparameter를 찾아보는 것도 한 방법이 될 수 있다.
```python
train_dataset = encoded_dataset['train'].shard(index=1, num_shards=10)
```
- 찾은 hyperparameter를 전체 데이터셋에 대해 학습할 때 적용하여 최종 모델을 학습시킨다.

In [52]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset['train'].shard(index=1, num_shards=10),
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.7cee10e8ea7ffa278f8be4b141000263f2b18795e5ef5e025352b2af6851f8fb
Model config BertConfig {
  "architectures": [
    "BertForPretraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads

Download file pytorch_model.bin:   0%|          | 13.0k/422M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.55k/2.55k [00:00<?, ?B/s]

Clean file training_args.bin:  39%|###9      | 1.00k/2.55k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/422M [00:00<?, ?B/s]

<br>

### 1.4.5 wandb 사용 설정

In [53]:
import wandb

wandb.login()

True

In [54]:
id = wandb.util.generate_id()
print(id)

7wx8exev


In [55]:
wandb.init(
    project='klue-test-hps',
    entity='zgotter',
    id=id,
    name='ynat',
    resume=True
)

<br>

### 1.4.6 hyperparameter search 수행

- `Trainer` 객체의 `hyperparameter_search` 메서드를 사용해 hyperparameter search를 수행한다.
- 입력 파라미터는 다음과 같다.

<br>

`hp_space`

- hyperparameter search를 수행할 딕셔너리를 반환하는 함수를 입력으로 받는다.
- 값을 설정하지 않을 경우 optuna의 기본값을 사용한다.

<br>

- optuna를 사용할 경우:
```python
def my_hp_space(trial):
    return {\
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'num_train_epochs': trial.suggest_int('num_train_epochs', 1, 5),
        'seed': trial.suggest_int('seed', 1, 40),
        'per_device_train_batch_size': trial.suggest_categorical('per_device_train_batch_size', [4, 8, 16, 32, 64])
    }
```

<br>

- ray를 사용할 경우:
```python
def my_hp_space_ray(trial):
    from ray import tune

    return {
        'learning_rate': tune.loguniform(1e-4, 1e-2),
        'num_train_epochs': tune.choice(range(1, 6)),
        'seed': tune.choice(range(1, 41)),
        'per_device_train_batch_size': tune.choice([4, 8, 16, 32, 64])
    }
```

<br>

`computive_objective`

- 최대화하거나 최소화할 목적 함수를 받는다.
- 기본값으로 모델의 `evaluate` 메서드에 의해 반환되는 metric 값(여기선 f1-score)를 사용한다.
```python
def my_objective(metrics):
    return metrics['eval_f1']
```

<br>

`n_trials`

- 테스트할 실험의 개수를 설정한다.
- 기본값 = 100

<br>

`direction`

- `computeive_objective` 값의 최적화의 방향을 정한다.
- 값의 종류
  - `minimize` (기본값)
  - `maximize`

In [56]:
best_run = trainer.hyperparameter_search(n_trials=5, direction='maximize')

[I 2021-09-24 02:28:51,128] A new study created in memory with name: no-name-65ec392d-023c-45a6-8350-8894f9bb9948
Trial:
loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.7cee10e8ea7ffa278f8be4b141000263f2b18795e5ef5e025352b2af6851f8fb
Model config BertConfig {
  "architectures": [
    "BertForPretraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LAB

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,F1
1,0.503200,0.612001,0.814582
2,0.377600,0.637294,0.839265
3,0.346200,0.741232,0.835995
4,0.309800,0.722539,0.845101
5,0.228500,0.717985,0.851052


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat-hps/run-0/checkpoint-1142
Configuration saved in test-klue/ynat-hps/run-0/checkpoint-1142/config.json
Model weights saved in test-klue/ynat-hps/run-0/checkpoint-1142/pytorch_model.bin
tokenizer config file saved in test-klue/ynat-hps/run-0/checkpoint-1142/tokenizer_config.json
Special tokens file saved in test-klue/ynat-hps/run-0/checkpoint-1142/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat-hps/run-0/checkpoint-2284
Configuration saved in test-klue

eval/f1,0.85105
eval/loss,0.71798
eval/runtime,15.4502
eval/samples_per_second,589.443
eval/steps_per_second,2.33
train/epoch,5.0
train/global_step,5710
train/learning_rate,0.0
train/loss,0.2285
train/total_flos,218089799608200.0
train/train_loss,0.41686


...

eval/f1,0.50983
eval/loss,1.62346
eval/runtime,15.3105
eval/samples_per_second,594.82
eval/steps_per_second,2.351
train/epoch,2.0
train/global_step,286
train/total_flos,102535692312000.0
train/train_loss,1.57203
train/train_runtime,99.142
train/train_samples_per_second,92.151


eval/f1,▁█
eval/loss,█▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁██
train/global_step,▁██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,F1
1,No log,1.520116,0.570573
2,No log,1.065122,0.728400
3,No log,0.862462,0.781736
4,1.049800,0.817062,0.786721


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat-hps/run-4/checkpoint-143
Configuration saved in test-klue/ynat-hps/run-4/checkpoint-143/config.json
Model weights saved in test-klue/ynat-hps/run-4/checkpoint-143/pytorch_model.bin
tokenizer config file saved in test-klue/ynat-hps/run-4/checkpoint-143/tokenizer_config.json
Special tokens file saved in test-klue/ynat-hps/run-4/checkpoint-143/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: guid, url, title, date.
***** Running Evaluation *****
  Num examples = 9107
  Batch size = 256
Saving model checkpoint to test-klue/ynat-hps/run-4/checkpoint-286
Configuration saved in test-klue/ynat-

<br>

### 1.4.7 hyperparameter search 결과 사용

- `hyperparameter_search` 메서드는 `BestRun 객체를 반환한다.
- 이 객체는 최대화된 objective의 값과 이때 선택된 hyperparameter를 포함한다.

In [57]:
best_run

BestRun(run_id='0', objective=0.85105193903787, hyperparameters={'learning_rate': 5.203061013978762e-06, 'num_train_epochs': 5, 'seed': 2, 'per_device_train_batch_size': 4})

<br>

### Appendix: optuna를 사용한 hyperparameter search code snippet

```python
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscoree_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
```

```python
class MemorySaverCallback(TrainerCallback):
    "A callback that deleted the folder in which checkpoints are saved, to save memory"
    def __init__(self, run_name):
        super(MemorySaverCallback, self).__init__()
        self.run_name = run_name

    def on_train_begin(self, args, state, control, **kwargs):
        print('Removing dirs...")
        if os.path.isidr(f'./{self.run_name}'):
            import shutil
            shutil.rmtree(f'./{self.run_name}')
        else:
            print('\n\nDirectory does not exists')
```

```python
training_args = TrainingArguments(
    RUN_NAME, 
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=False,
    overwrite_output_dir=True,
    save_strategy="no",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)
```

```python
trainer = Trainer(
    model_init=partial(MyNet, 2),
    args=training_args,
    train_dataset=training_opos.select(range(2000)),
    eval_dataset=validation_opos,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2), MemorySaverCallback(RUN_NAME)]
)
```

```python
def my_hp_space_optuna(trial):
    return {
        'learning_rate': trial.suggest_float('learning_rate', 2e-6, 2e-4, log=True),
        'warmup_steps':  trial.suggest_float('warmup_steps', 0., 0.9, step=0.3),
        'weight_decay':  trial.suggest_float('weight_decay', 1e-6, 1e-1)
    }
```

```python
def my_objective(metrics):
    return metrics['eval_f1']
```

```python
sa = trainer.hyperparameter_search(
    direction='maximize',
    n_trials=1,
    hp_space=my_hp_space_optuna,
    compute_objective=my_objective
)
```

<br>

## Reference

- [Text Classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=71pt6N0eIrJo)
- [HuggingFace Datasets Docs](https://huggingface.co/docs/datasets/index.html)
- [HuggingFace Transformers Docs](https://huggingface.co/transformers/index.html)
- [Using hyperparameter-search in Trainer](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/55)